![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use AutoAI RAG and Chroma to create a pattern and get information from `ibm-watsonx-ai` SDK documentation

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate the usage of IBM AutoAI RAG. The AutoAI RAG experiment conducted in this notebook uses data scraped from the `ibm-watsonx-ai` SDK documentation.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The learning goals of this notebook are:

- Create an AutoAI RAG job that will find the best RAG pattern based on provided data


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [RAG Optimizer definition](#definition)
- [RAG Experiment run](#run)
- [RAG Patterns comparison and testing](#comparison)
- [Historical runs](#runs)
- [Clean up](#cleanup)
- [Summary and next steps](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the required modules and dependencies

In [ ]:
!pip install -U 'ibm-watsonx-ai[rag]>=1.3.26' | tail -n 1

### Defining the watsonx.ai credentials
This cell defines the credentials required to work with the watsonx.ai Runtime service.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai api key (hit enter): "),
)

### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press **Create**
- Go to **Manage** tab
- Copy `Space GUID` into your env file or else enter it in the window which will show up after running below cell

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below

In [ ]:
import os

try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    space_id = input("Please enter your space_id (hit enter): ")

Create an instance of APIClient with authentication details.

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials=credentials, space_id=space_id)

<a id="definition"></a>

## RAG Optimizer definition

### Defining a connection to training data

Upload training data to a COS bucket and then define a connection to this file. This example uses the `Base` description from the [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/fm_model_inference.html) documentation.

The code in the next cell uploads training data to the bucket.

In [ ]:
import os
import requests

url = "https://ibm.github.io/watsonx-ai-python-sdk/base.html"

document_filename = "base.html"

response = requests.get(url)

response.raise_for_status()

if not os.path.isfile(document_filename):
    with open(document_filename, "w", encoding="utf-8") as file:
        file.write(response.text)

document_asset_details = client.data_assets.create(
    name=document_filename, file_path=document_filename
)

document_asset_id = client.data_assets.get_id(document_asset_details)
document_asset_id

Creating data asset...
SUCCESS


'68b540c6-1169-45a8-aac7-d3d1ebfe8e5f'

Define a connection to training data.

In [6]:
from ibm_watsonx_ai.helpers import DataConnection

input_data_references = [DataConnection(data_asset_id=document_asset_id)]

### Defining a connection to test data

Upload a `json` file that will be used for benchmarking to COS and then define a connection to this file. This example uses content from the [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/index.html) SDK documentation.

In [ ]:
benchmarking_data_IBM_page_content = [
    {
        "question": "How can you set or refresh user request headers using the APIClient class?",
        "correct_answer": "client.set_headers({'Authorization': 'Bearer <token>'})",
        "correct_answer_document_ids": ["base.html"],
    },
    {
        "question": "How to initialise Credentials object with api_key",
        "correct_answer": "credentials = Credentials(url = 'https://us-south.ml.cloud.ibm.com', api_key = '***********')",
        "correct_answer_document_ids": ["base.html"],
    },
]

The code in the next cell uploads testing data to the bucket as a `json` file.

In [ ]:
import json

test_filename = "benchmarking_data_Base.json"

if not os.path.isfile(test_filename):
    with open(test_filename, "w") as json_file:
        json.dump(benchmarking_data_IBM_page_content, json_file, indent=4)

test_asset_details = client.data_assets.create(
    name=test_filename, file_path=test_filename
)

test_asset_id = client.data_assets.get_id(test_asset_details)
test_asset_id

Creating data asset...
SUCCESS


'56411113-3ce3-44b8-b8ab-2d949c069496'

Define connection information to testing data.

In [9]:
test_data_references = [DataConnection(data_asset_id=test_asset_id)]

### RAG Optimizer configuration

Provide the input information for AutoAI RAG optimizer:
- `name` - experiment name
- `description` - experiment description
- `max_number_of_rag_patterns` - maximum number of RAG patterns to create
- `optimization_metrics` - target optimization metrics

In [ ]:
from ibm_watsonx_ai.experiment import AutoAI
from ibm_watsonx_ai.foundation_models.schema import (
    AutoAIRAGModelConfig,
    AutoAIRAGRetrievalConfig,
    AutoAIRAGLanguageConfig,
    AutoAIRAGGenerationConfig,
)

experiment = AutoAI(
    credentials=credentials,
    space_id=space_id,
)

foundation_model = AutoAIRAGModelConfig(
    model_id="ibm/granite-3-3-8b-instruct",
)

language_config = AutoAIRAGLanguageConfig(
    auto_detect=False,
)

generation_config = AutoAIRAGGenerationConfig(
    language=language_config,
    foundation_models=[foundation_model],
)

retrieval_config = AutoAIRAGRetrievalConfig(
    method="window",
    number_of_chunks=5,
    window_size=2,
)

chunking_config = {"method": "recursive", "chunk_size": 256, "chunk_overlap": 128}

rag_optimizer = experiment.rag_optimizer(
    name="AutoAI RAG - sample notebook",
    description="Experiment run in sample notebook",
    embedding_models=["ibm/slate-125m-english-rtrvr", "intfloat/multilingual-e5-large"],
    chunking=[chunking_config],
    generation=generation_config,
    retrieval=[retrieval_config],
    max_number_of_rag_patterns=3,
    optimization_metrics=[AutoAI.RAGMetrics.ANSWER_CORRECTNESS],
)

Configuration parameters can be retrieved via `get_params()`.

In [11]:
rag_optimizer.get_params()

{'name': 'AutoAI RAG - sample notebook',
 'description': 'Experiment run in sample notebook',
 'chunking': [{'method': 'recursive',
   'chunk_size': 256,
   'chunk_overlap': 128}],
 'embedding_models': ['ibm/slate-125m-english-rtrvr',
  'intfloat/multilingual-e5-large'],
 'max_number_of_rag_patterns': 3,
 'optimization_metrics': ['answer_correctness'],
 'generation': {'language': {'auto_detect': False},
  'foundation_models': [{'model_id': 'ibm/granite-3-3-8b-instruct'}]},
 'retrieval': [{'method': 'window', 'number_of_chunks': 5, 'window_size': 2}]}

<a id="run"></a>
## RAG Experiment run

Call the `run()` method to trigger the AutoAI RAG experiment. You can either use interactive mode (synchronous job) or background mode (asynchronous job) by specifying `background_mode=True`.

In [ ]:
run_details = rag_optimizer.run(
    input_data_references=input_data_references,
    test_data_references=test_data_references,
    background_mode=False,
)



##############################################

Running '85cecb9b-1c21-4503-91a4-fc14d43ff8fa'

##############################################


pending.........
running.........
completed
Training of '85cecb9b-1c21-4503-91a4-fc14d43ff8fa' finished successfully.


You can use the `get_run_status()` method to monitor AutoAI RAG jobs in background mode.

In [13]:
rag_optimizer.get_run_status()

'completed'

<a id="comparison"></a>
## Comparison and testing of RAG Patterns

You can list the trained patterns and information on evaluation metrics in the form of a Pandas DataFrame by calling the `summary()` method. You can use the DataFrame to compare all discovered patterns and select the one you like for further testing.

In [14]:
summary = rag_optimizer.summary()
summary

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.method,chunking.chunk_size,chunking.chunk_overlap,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,retrieval.hybrid_ranker,generation.model_id
Pattern_Name,,,,,,,,,,,,
Pattern2,0.7083,0.2422,1.0,recursive,256,128,intfloat/multilingual-e5-large,cosine,window,5,,ibm/granite-3-3-8b-instruct
Pattern1,0.5833,0.3195,1.0,recursive,256,128,ibm/slate-125m-english-rtrvr,cosine,window,5,,ibm/granite-3-3-8b-instruct


Additionally, you can pass the `scoring` parameter to the summary method, to filter RAG patterns starting with the best.

```python
summary = rag_optimizer.summary(scoring="faithfulness")
```

In [15]:
rag_optimizer.get_run_details()

{'entity': {'hardware_spec': {'id': 'a6c4923b-b8e4-444c-9f43-8a7ec3020110',
   'name': 'L'},
  'input_data_references': [{'location': {'href': '/v2/assets/68b540c6-1169-45a8-aac7-d3d1ebfe8e5f?space_id=6f7e632c-8af4-4626-9fe2-c7128b43d32e',
    'id': '68b540c6-1169-45a8-aac7-d3d1ebfe8e5f'},
   'type': 'data_asset'}],
  'parameters': {'constraints': {'chunking': [{'chunk_overlap': 128,
     'chunk_size': 256,
     'method': 'recursive'}],
   'embedding_models': ['ibm/slate-125m-english-rtrvr',
    'intfloat/multilingual-e5-large'],
   'generation': {'foundation_models': [{'model_id': 'ibm/granite-3-3-8b-instruct'}],
    'language': {'auto_detect': False}},
   'max_number_of_rag_patterns': 3,
   'retrieval': [{'method': 'window',
     'number_of_chunks': 5,
     'window_size': 2}]},
   'optimization': {'metrics': ['answer_correctness']},
   'output_logs': True},
  'results': [{'context': {'iteration': 0,
    'max_combinations': 2,
    'rag_pattern': {'composition_steps': ['model_selection

### Get selected pattern

Get the RAGPattern object from the RAG Optimizer experiment. By default, the RAGPattern of the best pattern is returned.

In [ ]:
best_pattern_name = summary.index.values[0]
print("Best pattern is:", best_pattern_name)

best_pattern = rag_optimizer.get_pattern(pattern_name="Pattern1")

Best pattern is: Pattern2


The pattern details can be retrieved by calling the `get_pattern_details` method:

```python
rag_optimizer.get_pattern_details(pattern_name='Pattern2')
```

Query the RAGPattern locally, to test it.

In [17]:
from ibm_watsonx_ai.deployments import RuntimeContext

runtime_context = RuntimeContext(api_client=client)
inference_service_function = best_pattern.inference_service(runtime_context)[0]

In [ ]:
question = "How to add Task Credentials?"

context = RuntimeContext(
    api_client=client,
    request_payload_json={"messages": [{"role": "user", "content": question}]},
)

inference_service_function(context)

{'body': {'choices': [{'index': 0,
   'message': {'role': 'system',
    'content': 'To add task credentials in IBM Watsonx.ai, you would use the Credentials class. Here\'s a general guideline on how to create and set credentials:\n\n1. **Using an API Key:**\n\n```python\nfrom ibm_watsonx_ai import Credentials\n\ncredentials = Credentials(\n   url = "https://us-south.ml.cloud.ibm.com",\n   api_key = \'IAM_API_KEY\'\n)\n```\n\n2. **Using a Token:**\n\n```python\ncredentials = Credentials(\n   url = "https://us-south.ml.cloud.ibm.com",\n   token = "***********"\n)\n```\n\n3. **Using Username and Password:**\n\n```python\ncredentials = Credentials(\n   url = "<URL>",\n   username = "<USERNAME>",\n   password = "<PASSWORD>",\n   instance_id = "openshift"\n)\n```\n\n4. **Using Username and API Key:**\n\n```python\ncredentials = Credentials(\n   url = "<URL>",\n   username = "<USERNAME>",\n   api_key = IAM_API_KEY,\n   instance_id = "openshift"\n)\n```\n\nOnce you have created the Credentials

### Deploy RAGPattern

Deployment is done by storing the defined RAG function and then by creating a deployed asset.

In [ ]:
deployment_details = best_pattern.inference_service.deploy(
    name="AutoAI RAG deployment - ibm_watsonx_ai documentataion",
    space_id=space_id,
    deploy_params={"tags": ["wx-autoai-rag"]},
)



######################################################################################

Synchronous deployment creation for id: '621f18df-cb6c-49d7-84ba-83e148bf875c' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
......
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='1d5d0cbb-3aa5-4acd-b998-c46f4b210938'
-----------------------------------------------------------------------------------------------




### Test the deployed function

RAG service is now deployed in our space. To test our solution we can run the cell below. Questions have to be provided in the payload. Their format is provided below.

In [ ]:
deployment_id = client.deployments.get_id(deployment_details)

payload = {"messages": [{"role": "user", "content": question}]}
score_response = client.deployments.run_ai_service(deployment_id, payload)

In [21]:
print(score_response["choices"][0]["message"]["content"])

To add task credentials in Python while working with IBM watsonx.ai, you would use the `Credentials` class provided by the `ibm_watsonx_ai` package. This class helps encapsulate passed credentials and additional parameters. Here's an example on how you can do this:

First, you need to import the necessary classes and modules:

```python
from ibm_watsonx_ai import Credentials, APIClient
```

Next, create a task token. This token is used to authenticate the client:

```python
task_token = context.generate_token()
```

Now, instantiate the `Credentials` object using the task token (or API key/token as per your use case):

```python
# Using task token
credentials = Credentials(
   url = "https://us-south.ml.cloud.ibm.com",
   token = task_token
)
```

Now, you can create an instance of the `APIClient` using the credentials object:

```python
client = APIClient(credentials)
```

You can create a function to encapsulate your operations with this client and handle token generation within it. 

In [22]:
score_response["choices"][0]["message"]["content"]

'To add task credentials in Python while working with IBM watsonx.ai, you would use the `Credentials` class provided by the `ibm_watsonx_ai` package. This class helps encapsulate passed credentials and additional parameters. Here\'s an example on how you can do this:\n\nFirst, you need to import the necessary classes and modules:\n\n```python\nfrom ibm_watsonx_ai import Credentials, APIClient\n```\n\nNext, create a task token. This token is used to authenticate the client:\n\n```python\ntask_token = context.generate_token()\n```\n\nNow, instantiate the `Credentials` object using the task token (or API key/token as per your use case):\n\n```python\n# Using task token\ncredentials = Credentials(\n   url = "https://us-south.ml.cloud.ibm.com",\n   token = task_token\n)\n```\n\nNow, you can create an instance of the `APIClient` using the credentials object:\n\n```python\nclient = APIClient(credentials)\n```\n\nYou can create a function to encapsulate your operations with this client and han

<a id="runs"></a>
## Historical runs

In this section you learn to work with historical RAG Optimizer jobs (runs).

To list historical runs use the `list()` method and provide the `'rag_optimizer'` filter.

In [ ]:
experiment.runs(filter="rag_optimizer").list()

In [ ]:
run_id = run_details["metadata"]["id"]
run_id

'85cecb9b-1c21-4503-91a4-fc14d43ff8fa'

### Get executed optimizer's configuration parameters

In [25]:
experiment.runs.get_rag_params(run_id=run_id)

{'name': 'AutoAI RAG - sample notebook',
 'description': 'Experiment run in sample notebook',
 'chunking': [{'chunk_overlap': 128,
   'chunk_size': 256,
   'method': 'recursive'}],
 'embedding_models': ['ibm/slate-125m-english-rtrvr',
  'intfloat/multilingual-e5-large'],
 'max_number_of_rag_patterns': 3,
 'generation': {'foundation_models': [{'model_id': 'ibm/granite-3-3-8b-instruct'}],
  'language': {'auto_detect': False}},
 'retrieval': [{'method': 'window', 'number_of_chunks': 5, 'window_size': 2}],
 'optimization_metrics': ['answer_correctness']}

### Get historical rag_optimizer instance and training details

In [26]:
historical_opt = experiment.runs.get_rag_optimizer(run_id)

### List trained patterns for selected optimizer

In [27]:
historical_opt.summary()

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.method,chunking.chunk_size,chunking.chunk_overlap,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,retrieval.hybrid_ranker,generation.model_id
Pattern_Name,,,,,,,,,,,,
Pattern2,0.7083,0.2422,1.0,recursive,256,128,intfloat/multilingual-e5-large,cosine,window,5,,ibm/granite-3-3-8b-instruct
Pattern1,0.5833,0.3195,1.0,recursive,256,128,ibm/slate-125m-english-rtrvr,cosine,window,5,,ibm/granite-3-3-8b-instruct


<a id="cleanup"></a>
## Clean up

To delete the current experiment, use the `cancel_run` method.

**Warning:** Be careful: once you delete an experiment, you will no longer be able to refer to it.

In [28]:
rag_optimizer.cancel_run(hard_delete=True)

'SUCCESS'

To delete the deployment, use the `delete` method. 

**Warning:** Keeping the deployment active may lead to unnecessary consumption of Compute Unit Hours (CUHs).

In [29]:
client.deployments.delete(deployment_id)

'SUCCESS'

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to use `ibm-watsonx-ai` to run AutoAI RAG experiments. 

 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Michał Steczko**, Software Engineer watsonx.ai

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.